In [5]:
import typing
import bittensor as bt
import pydantic

class Store(bt.Synapse):
    class Config:
        validate_assignment = True

    def deserialize(self) -> str:
        return self.zk_proof

    # Query values
    content: typing.Optional[object] = pydantic.Field(..., allow_mutation=False)
    content_hash: str = pydantic.Field(..., allow_mutation=False)
    pubkey: str = pydantic.Field(..., allow_mutation=False)
    signature: str = pydantic.Field(..., allow_mutation=False)

    # Return values
    stored: typing.Optional[bool] = False 
    error_messgae: typing.Optional[str] = ''

class Retrieve(bt.Synapse):
    class Config:
        validate_assignment = True

    def deserialize(self) -> str:
        return self.verified

    # Query values
    content_hash: str = pydantic.Field(..., allow_mutation=False)

    # Return values
    in_registry: typing.Optional[bool] = False
    miner_signature: typing.Optional[str] = ''
    miner_pubkey: typing.Optional[str] = ''

In [9]:
import hashlib
import secrets
import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm

def hash256(content: str) -> str:
    return hashlib.sha256(content.encode()).hexdigest()

def generate_password(seed, length=12):
    # Convert the seed to string
    seed_str = str(seed)
    
    # Hash the string representation using SHA-256
    hashed = hashlib.sha256(seed_str.encode()).hexdigest()
    
    # Use the hash to generate a password
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*()-_=+"
    password = ''.join(secrets.choice(alphabet) for i in range(length))
    
    return password

def sign_content_with_new_keypair(content: str, passphrase: str = None, seed: int = 1337) -> (str, str):
    """
    Sign the content using the private key.

    Args:
    - content (str): The content to sign.
    - passphrase (str): The passphrase for the private key.

    Returns:
    - tuple: The cleartext signature and the public key.
    """
    # Generate a primary key
    key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 4096)

    # Add a user ID to the key
    uid = pgpy.PGPUID.new('Test User', email='test@example.com')
    key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
                hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA512],
                ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES128],
                compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])

    # TODO: use a real passphrase and get it from the env variables or terminal input
    # For not just generate a throw-away passphrase
    if passphrase is None:
        passphrase = generate_password(seed)

    # Protect the key with the passphrase
    key.protect(passphrase, SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)

    # Unlock the key to sign data
    with key.unlock(passphrase):
        # Create a cleartext signature
        cleartext_signature = pgpy.PGPMessage.new(content, cleartext=True)
        cleartext_signature |= key.sign(cleartext_signature, hash=HashAlgorithm.SHA256)

    return str(cleartext_signature), str(key.pubkey)

def verify_pgpy_with_content(content: str, signature: str, pubkey: str) -> bool:
    """
    Verify the cleartext signature of the content using the provided public key.

    Args:
    - signature (str): The cleartext signature string.
    - content (str): The content string.
    - pubkey (str): The public key string.

    Returns:
    - bool: True if the signature is valid, False otherwise.
    """
    # Load the public key
    public_key, _ = pgpy.PGPKey.from_blob(pubkey)

    # Load the signature
    signature_obj = pgpy.PGPSignature.from_blob(signature)

    # Convert the content to bytes if it's not
    if not isinstance(content, bytes):
        content = content.encode('utf-8')

    # Verify the signature using the content and public key
    return public_key.verify(content, signature_obj)

sign = sign_content_with_new_keypair
verify = verify_pgpy_with_content
hash = hash256


In [10]:
registry = {}

class SignatureMismatchError(Exception):
    pass

class ContentHashMismatchError(Exception):
    pass

def store( synapse: Store ) -> Store:
    try:
        # Check content_hash against the content
        local_content_hash = hash( synapse.content )
        # If it matches, check the signature against the pubkey
        if synapse.content_hash == local_content_hash:
            if verify( synapse.content, synapse.signature, synapse.pubkey ):
                # # If it matches, generate a signature of the content signed with the miner key
                # store the content has as key and the (miner_signature, pubkey) pairs in the database
                miner_signature, miner_pubkey = sign( synapse.content_hash )
                registry[ synapse.content_hash ] = ( miner_signature, miner_pubkey )
                stored = True
                # Optimistically store (no need to send back the signature until verify step)
            else:
                # If it doesn't match, return an error. Attempted to store invalid content.
                stored = False
                raise SignatureMismatchError("Signature is not valid with provided pubkey!")
        else:
            # If it doesn't match, return an error.
            stored = False
            raise ContentHashMismatchError("Content hash mismatch, data tampered with!")
    except SignatureMismatchError as e:
        synapse.error_message = e
    except ContentHashMismatchError as e:
        synapse.error_message = e
    except Exception as e:
        synapse.error_message = "Unknown error occured."
    finally:
        # return the filled synapse
        synapse.stored = stored
        return synapse

In [11]:
content = "This is some content"
content_hash = hash( content )
signature, pubkey = sign_content_with_new_keypair( content )

/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:194: CryptographyDeprecationWarning: CAST5 has been deprecated
  SymmetricKeyAlgorithm.CAST5: algorithms.CAST5,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:195: CryptographyDeprecationWarning: Blowfish has been deprecated
  SymmetricKeyAlgorithm.Blowfish: algorithms.Blowfish,


In [12]:
syn = Store(content=content, content_hash=content_hash, pubkey=pubkey, signature=signature)
syn

Store(content='This is some content', content_hash='d68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGT+Qs8BEACv60lvRSb+mGdEcDB8U8cuvtXF2ZrDEumf3ay4k3KS4ilr/J4c\nqgk0NM7NeZ4/1EiPL4mifYuoPS4nFhLF3zgs34zow1K5Z1cQFbToNXXm7z5qsecX\nbMgSzQTy7ha6zW2Vkzxaw6L5LgflP+gqKzmnNlstUP6ZtkAcsB+VbaeIprQnmcpD\nkm6o+FswK5PUVrWORgWqpyam+GtUKd68huADR1qdHJ2odAO1ew53Xdsl14T9vbri\nEmRArW6lTfp2734xHUgpP5Ki9QNoiKKdeYNEeuh8gEejt/uTQj5IxoAxdeRvmbZ9\nQt27MkdZ4vPeLRRQmYebY27vIj3LVHiOVCaCQAq0vq2IP8jHbvygzPEEdnZgWaZm\nN4e/jp4nWb8FrmWccfy7iA5Zk5fvjV9x3DvA+S8ejpRx6GgFsjeKP1jlsc13V1sN\nGX3gTdgkdnJ5RpyzV19kmWiVJ1su1AOxeMqp2XKUbSgBT7ffUC3QNnOZ76bS1dxw\niRMmxGYGnqmScLBiWJR/MKaJI0HB1wuLj+uCAFVR4oHRnfcYZQ/DOUMA+7pG5HE2\nMjVSYLBS6pDo/8dQvXH5lRRU6fgYO2GkJFVxAGMiI81lr7dMpY9nUS8PAM8/Sesr\nzyAYJYOhpoMGceO9oLADYCF7Z4LlUnlBmK7EAP/kEAeDSEy3oes5Q06UEwARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJk/kLQAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBFaTNuY0KWcSB7ykp

In [13]:
filled_syn = store( syn )
filled_syn

/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2389: UserWarning: TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!
  warnings.warn("TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!")
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2406: UserWarning: TODO: Revocation checks are not yet implemented!!!
  warnings.warn("TODO: Revocation checks are not yet implemented!!!")
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2407: UserWarning: TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!
  warnings.warn("TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!")


Store(content='This is some content', content_hash='d68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGT+Qs8BEACv60lvRSb+mGdEcDB8U8cuvtXF2ZrDEumf3ay4k3KS4ilr/J4c\nqgk0NM7NeZ4/1EiPL4mifYuoPS4nFhLF3zgs34zow1K5Z1cQFbToNXXm7z5qsecX\nbMgSzQTy7ha6zW2Vkzxaw6L5LgflP+gqKzmnNlstUP6ZtkAcsB+VbaeIprQnmcpD\nkm6o+FswK5PUVrWORgWqpyam+GtUKd68huADR1qdHJ2odAO1ew53Xdsl14T9vbri\nEmRArW6lTfp2734xHUgpP5Ki9QNoiKKdeYNEeuh8gEejt/uTQj5IxoAxdeRvmbZ9\nQt27MkdZ4vPeLRRQmYebY27vIj3LVHiOVCaCQAq0vq2IP8jHbvygzPEEdnZgWaZm\nN4e/jp4nWb8FrmWccfy7iA5Zk5fvjV9x3DvA+S8ejpRx6GgFsjeKP1jlsc13V1sN\nGX3gTdgkdnJ5RpyzV19kmWiVJ1su1AOxeMqp2XKUbSgBT7ffUC3QNnOZ76bS1dxw\niRMmxGYGnqmScLBiWJR/MKaJI0HB1wuLj+uCAFVR4oHRnfcYZQ/DOUMA+7pG5HE2\nMjVSYLBS6pDo/8dQvXH5lRRU6fgYO2GkJFVxAGMiI81lr7dMpY9nUS8PAM8/Sesr\nzyAYJYOhpoMGceO9oLADYCF7Z4LlUnlBmK7EAP/kEAeDSEy3oes5Q06UEwARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJk/kLQAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBFaTNuY0KWcSB7ykp

In [27]:
# Store another
content = "This is some extra content"
content_hash = hash( content )
signature, pubkey = sign_content_with_new_keypair( content )

syn2 = Store(content=content, content_hash=content_hash, pubkey=pubkey, signature=signature)
syn2

/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:194: CryptographyDeprecationWarning: CAST5 has been deprecated
  SymmetricKeyAlgorithm.CAST5: algorithms.CAST5,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:195: CryptographyDeprecationWarning: Blowfish has been deprecated
  SymmetricKeyAlgorithm.Blowfish: algorithms.Blowfish,


Store(content='This is some extra content', content_hash='ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGT+Q0IBEACMmX/xIvlF+6oFmbYhQe3SUYD4SlJQ/mVaEtYLuLsHa0Wl0uSd\nfWpM1umyhw9VtS6U8IdTnT9YFHne8yw8wfQbcd39WgzD84S7Z4BRY3zb63BHDyPO\nphaKbEf7kVFzD1Ognfd/YJWdmoBVChNe71GMZdt9qYMY17Yfa6OMnNgS/o+3amKv\nVUrZGOnJNWyhKaJDqU8qW/QlSddGOZ2cDULStsLQO6VwtUMq7E7ZChDXKBhWTv/y\n9RISZ9KkU5onyXP8PT/q8R+DNsXRI+iamdGKzIEgjJJhHPQwuKX5o0/p6Q3bCqlr\nqU2jC86BM9CFQ00wDS3hS92xFmuh9+1CxnbyguJTh+YqD9BDN8kEvK7a/UVrq5vu\n9RnUXkXBetUvTyOu5op7zFH8HBQb9VZSPnY2uB3qVgaprnSv9kylS1Osc1V8hEbd\nUsVAzqE+tjP9JFTRSg7DEVWtdNitzEUZKua1kgwyvYF2AsAhjd1YlaTEDxKwHfLX\nGb6Usn9kwhX+pblKl8T45fcOmQWyhR/JqahZVzQxJPtKfKBCtaOligOvr89jIiOT\nWxw5w6dIheKMWY/9GgKsGK+f5KpgzRbky7wVEU8Ai4WdWwGii/6GyZ8QVtowyLHD\nkxPFqCL4/rQQIJTYQa4F6gY7daLIKcEqWBGVBTF3qfxFJjB/qPe14FEtTQARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJk/kNDAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBP8r1pMGGbV

In [28]:
filled_syn2 = store( syn2 )
filled_syn2

/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2389: UserWarning: TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!
  warnings.warn("TODO: Self-sigs verification is not yet working because self-sigs are not parsed!!!")
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2406: UserWarning: TODO: Revocation checks are not yet implemented!!!
  warnings.warn("TODO: Revocation checks are not yet implemented!!!")
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/pgp.py:2407: UserWarning: TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!
  warnings.warn("TODO: Flags (s.a. `disabled`) checks are not yet implemented!!!")
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/home/jason/miniconda3/envs/rev/lib/python3.10/site-packages/pgpy/constants

Store(content='This is some extra content', content_hash='ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e', pubkey='-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBGT+Q0IBEACMmX/xIvlF+6oFmbYhQe3SUYD4SlJQ/mVaEtYLuLsHa0Wl0uSd\nfWpM1umyhw9VtS6U8IdTnT9YFHne8yw8wfQbcd39WgzD84S7Z4BRY3zb63BHDyPO\nphaKbEf7kVFzD1Ognfd/YJWdmoBVChNe71GMZdt9qYMY17Yfa6OMnNgS/o+3amKv\nVUrZGOnJNWyhKaJDqU8qW/QlSddGOZ2cDULStsLQO6VwtUMq7E7ZChDXKBhWTv/y\n9RISZ9KkU5onyXP8PT/q8R+DNsXRI+iamdGKzIEgjJJhHPQwuKX5o0/p6Q3bCqlr\nqU2jC86BM9CFQ00wDS3hS92xFmuh9+1CxnbyguJTh+YqD9BDN8kEvK7a/UVrq5vu\n9RnUXkXBetUvTyOu5op7zFH8HBQb9VZSPnY2uB3qVgaprnSv9kylS1Osc1V8hEbd\nUsVAzqE+tjP9JFTRSg7DEVWtdNitzEUZKua1kgwyvYF2AsAhjd1YlaTEDxKwHfLX\nGb6Usn9kwhX+pblKl8T45fcOmQWyhR/JqahZVzQxJPtKfKBCtaOligOvr89jIiOT\nWxw5w6dIheKMWY/9GgKsGK+f5KpgzRbky7wVEU8Ai4WdWwGii/6GyZ8QVtowyLHD\nkxPFqCL4/rQQIJTYQa4F6gY7daLIKcEqWBGVBTF3qfxFJjB/qPe14FEtTQARAQAB\nzRxUZXN0IFVzZXIgPHRlc3RAZXhhbXBsZS5jb20+wsGGBBMBCAAwBQJk/kNDAhsO\nAwsJBwMVCAoEFgIBAAIeARYhBP8r1pMGGbV

In [29]:
# Validator must check the signature against the pubkey from miner
def retrieve( synapse: Retrieve ) -> Retrieve:
    if synapse.content_hash in registry:
        # If the content has is in the database, check the signature against the pubkey
        miner_signature, miner_pubkey = registry[ synapse.content_hash ]
        in_registry = True
    else:
        # If it isn't, return 
        miner_signature = None
        miner_pubkey = None
        in_registry = False
    # Fill values
    # TODO: Validator will use this to verify after receiving the filled synapse
    synapse.miner_signature = miner_signature
    synapse.miner_pubkey = miner_pubkey
    synapse.in_registry = in_registry
    # return the filled synapse
    return synapse


In [30]:
syn = Retrieve(content_hash=content_hash)
syn

Retrieve(content_hash='ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e', in_registry=False, miner_signature='', miner_pubkey='')

In [31]:
ret_syn = retrieve( syn )
ret_syn

Retrieve(content_hash='ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e', in_registry=True, miner_signature='-----BEGIN PGP SIGNED MESSAGE-----\nHash: SHA256\n\nee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e\n-----BEGIN PGP SIGNATURE-----\n\nwsFzBAEBCAAdBQJk/kNTFiEEuqCM0takNwD/+XhhpSlcdBaYPJUACgkQpSlcdBaY\nPJVefBAAqcTodSLyEX5cSGRFqwD2QifCECXM3ZzMpTfxVUwV0hTc2LvIwdky4+0B\nCKpKFfuj84p+CZSLD8LQjNX6BYShu6zZdf8Umid8MCYhYLjDBrli57S9NwBiLrxa\ndd+Sfbuy9W0WkPMjXepLVHfzUmQw7iwJfp4ysGFdKaTlgMhPbfRvITwODHwWqYEw\ntCbvvpS/oWMp/UBOXGFQEMCIg6VtUq3/kT1hXeikSw9G4Lr7bnR8APMiDV234XrV\nunSSzc3DPeonYe/ivhej9I8GVPSKSekmhxPbfwn/Cf1OHMnlT/MkiQSu1RE/DuRU\nlkVFoqh7m/aedWgWoTFS2YOHEQ+HrNccAIKToN1dIbAZJcFi+2UmA7hBAFLjuFml\nFhzQckpxFnHqEQyzmEUN6KQFWOUuzH/uCIwwbZUn9PiPnaUnXgdZbgtFj4d3ceM5\nfmZiicxEN6xNwqJtD/1s/nYjQVdwUU9ET5nPyPRMrKdAlggn7yRSEhtGDFDaGqaf\nbB3ELXyZpQmM6M8gAx0BnbEbtXQTZmxNW0wZReu71j3/xz4lAaJ4G4cBqe0XTTcd\n3jfe7vOzfX5Asf4UsN4+LosRvW+YqOKVtIUANATiYUGJI4pySeoj19IQj98hsBKY

In [32]:
# Validate the miner sig and pubkey against the content hash
verify( content_hash, ret_syn.miner_signature, ret_syn.miner_pubkey )

<SignatureVerification(True)>

In [33]:
class GetSize(bt.Synapse):
    class Config:
        validate_assignment = True

    def deserialize(self) -> str:
        return self.size

    # Return values
    registry_size: typing.Optional[int] = 0

syn = GetSize()
syn

GetSize(registry_size=0)

In [34]:
def get_size( synapse: GetSize ) -> GetSize:
    # Fill values
    synapse.registry_size = len( registry )
    # return the filled synapse
    return synapse

In [38]:
sizes = [get_size( syn ).registry_size]
sizes

[2]

In [53]:
import torch
maxval = max(sizes)
maxval
random_indices = torch.randperm(maxval)

alpha = 0.5  # for example, to select 50% of the indices
n = int(alpha * maxval)

selected_indices = random_indices[:n]
selected_indices

tensor([0])

In [56]:
# TODO: The Retrieve synapse should send indices rather than hashes...

class Verify(bt.Synapse):
    class Config:
        validate_assignment = True

    def deserialize(self) -> str:
        return self.verified

    # Query values
    registry_indices: typing.List[int] = pydantic.Field(..., allow_mutation=False)

    # Return values
    all_in_registry: typing.Optional[bool] = False
    all_verified: typing.Optional[bool] = False
    miner_signatures: typing.Optional[str] = ''
    miner_pubkeys: typing.Optional[str] = ''

vsyn = Verify( registry_indices = selected_indices.tolist() )
vsyn

Verify(registry_indices=[0], all_in_registry=False, all_verified=False, miner_signatures='', miner_pubkeys='')

In [60]:
keys = list(registry.keys())
keys

['d68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12',
 'ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e']

In [65]:
import numpy as np
arrkeys = np.array(keys)
arrkeys

array(['d68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12',
       'ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e'],
      dtype='<U64')

In [69]:
selected_indices = [0,1]
selected_keys = arrkeys[selected_indices]
selected_keys

array(['d68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12',
       'ee2ebb1b9ddd4bb04ed6e5345ea9592579364ae7f89d853327ebab6f70832c1e'],
      dtype='<U64')

In [74]:
# TODO: THESE SOULD BE PAIRS OF (SIGNATURE, PUBKEY) so we can easily compare them

miner_data = {'hashes': [], 'pubkeys': []}
for i in selected_keys:
    miner_hash, miner_pubkey = registry[i]
    hashes.append(miner_hash)
    pubkeys.append(miner_pubkey)

miner_data['hashes'] = hashes
miner_data['pubkeys'] = pubkeys
miner_data

{'hashes': ['-----BEGIN PGP SIGNED MESSAGE-----\nHash: SHA256\n\nd68e560efbe6f20c31504b2fc1c6d3afa1f58b8ee293ad3311939a5fd5059a12\n-----BEGIN PGP SIGNATURE-----\n\nwsFzBAEBCAAdBQJk/kLVFiEEWIkrFMwI8nKo+cJ1eoGTvrLyZyYACgkQeoGTvrLy\nZyaQyBAAtnoAIHBZVzGmMQ7B+RgsHd4yB8pKjr5ozsZFEhAaYW1/oMuvACdBN4Jk\nSIKgQTcUV9t65SMKrCBLR9644YDeeI6RZ1qGqiNM/QHlky2YJqXew1OMmTVi7OLW\n5a8ENSjArXBCqKiHVuu4ymeOlfAhjUwtg0sNTyDW9H0MtCIVq1gvVpo5YdpMezpU\n1nAn7WxPqirQSaVXFcxgnTLufgh5kekr4H8DRa5lg9+KQI29/a90B+cxjE1whQMz\nYUO4kam+dSwnYVNDU7PB8EgiB5Y39vmD0p0HTPtXjZrq8PUpgSb2z6zNS6ZPlE+C\nw3jHo8a9M7kRc5nl1eMCbk4Xydp4bpWeXIMMryGdSiOVaiLueK09sVPBP95NmWWd\nJ7jg5z1UmB/B7/xcwCUYm7w8Gya+nChqweTxBwQTrXBg1cRJnjrrPJ3h7ScWPm1a\nyTIBsgTcUNpgZW2UhOCIRnsYU9lwo1JztsZ2usRdwVmydtWYMExcFuRcy2pRrTsT\nFavqNNxQUssg6FRVhSIZWxCxBng8yW/PQKdkln3f/g3f/RfL1kAOy7X9oWg6/Dxs\n/rOQqHxGHpF7BGOp7Niq7Wqtdt7j0y7GMiVzxaCblvrDW16ihAYqFLFoE46iK1JL\nhKhm7S5a/Fr8N350EO05bETsz3g/cN8iyLYDnGgL7AMWC/bmTtI=\n=UA40\n-----END PGP SIGNATURE-----\n',
  '-----BEGIN PGP

In [ ]:
def verify_random_indices( indices: typing.List[int] ):
    keys = torch.tensor(list(registry.keys()))
    keys[indices]

In [ ]:
verify_random_indices( indices )

In [16]:
# modulo index length to ensure it's in range
# Validator side
import numpy as np
idxs = np.random.randint(0, 1000, 10)
idxs
# Miner recieves list of indices

array([413, 476, 251, 548, 792,  12,  55, 675, 212, 673])

In [17]:
# Miner side
registry = np.array(list(range(100)))
idx = (idxs ^ len(registry)) % len(registry)
idx

array([ 5, 40, 59, 76, 92,  4, 83, 11, 76,  9])